holdout
- 모델의 성능을 측정하고자 할 때 학습할 때 사용한 데이터로 검증하지 않기 위해 train-test 데이터를 분리한다.

- 데이터를 랜덤하게 두 분류로 분리하여 교차검증을 실시

사이킷런의 model_selection 모듈은 학습 데이터와 테스트 데이터 세트를 분리하거나 교차 검증 분할 및 평가, Estimator의 하이퍼 파라미터를 튜닝하기 위한 다양한 함수와 클래스 제공  

train_test_split() : 전체 데이터를 학습 데이터와 테스트 데이터 세트로 분리

훈련데이터와 테스트데이터는 중복되면 안된다.

In [24]:
import numpy as np
import pandas as pd

### 학습 데이터로만 학습하고 예측하는 경우

정확도가 100%가 나와 아주 의심스럽다. 예측 결과가 100% 정확도를 보이는 이유는 이미 학습한 학습 데이터 세트를 기반으로 예측했기 때문이다.     
따라서, 예측을 수행하는 데이터 세트는 학습을 수행한 학습용 데이터 세트가 아닌 전용의 테스트 데이터 세트여야 한다.

In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

iris = load_iris()
dt_clf = DecisionTreeClassifier()
train_data = iris.data
train_label = iris.target
dt_clf.fit(train_data, train_label)

# 학습 데이터 셋으로 예측 수행
pred = dt_clf.predict(train_data)
print('예측 정확도:',accuracy_score(train_label,pred))

예측 정확도: 1.0


#### 과적합에 안걸리게 하는 이슈

과적합 : 모델이 학습 데이터에만 과도하게 최적화되어 실제 예측을 다른 데이터로 수행할 경우에는 예측 성능이 과도하게 떨어지는 현상

- 훈련데이터와 테스트데이터는 중복이 없어야 한다. -> duplicated
- 데이터를 나눈 이후의 target값(0,1)의 비율이 원데이터에서 target값의 비율과 동일해야한다. -> stratify

### 학습/테스트 데이터 세트 분리해서 학습하고 예측하는 경우


### train_test_split 특징
- 첫번째 파라미터로 피처 데이터 세트, 두번째 파라미터로 레이블 데이터 세트를 입력받음.
    array를 1개 넣으면 값이 2개 반환    
    array를 2개 넣으면 값이 4개 반환   

- 관례상 다음과 같이 할당

    X_train : 훈련데이터   
    X_test : 테스트데이터  
    y_train : 학습데이터에 적용되는 target데이터(정답)  
    y_test : 테스트데이터에 적용되는 target데이터(정답)
    



#### train_test_split의 파라미터 설정  
- test_size : 디폴트값으로 전체 데이터에서 테스트 데이터 세트의 크기를 얼마로 할것인지 지정. 디폴트 25%  
    즉, 학습:테스트 = 75:25 비율이 디폴트  
- shuffle : 데이터를 분리하기 전에 데이터를 미리 섞을지를 결정. 디폴트 True. 데이터를 분산시켜 좀 더 효율적인 학습 및 테스트 데이터 셋을 만든다  
- random_state : 호출할 때마다 동일한 학습/테스트용 데이터 세트를 생성하기 위해 주어지는 난수 값이다. 지정하지 않으면 수행할 때마다 다른 학습/테스트 데이터를 생성한다.

In [16]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split


iris_data = load_iris()

# 학습:테스트 = 70:30
# 학습을 위해 난수 값 지정해 수행할때마다 동일한 결과로 고정
X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target, 
                                                    test_size=0.3, random_state=121)

#### 만약 train에는 정답이 없고 test에만 정답이 모두 들어가도록 쪼개지는 경우는 학습이 안되는 문제가 생길 수 있다.

예를 들면 이런 target값들중 0이 모두 train set에만 포함되고, test set에 없다면 0은 예측할 수 없다.

비율을 유지하며 쪼개졌는지 확인, target값들 종류별로 개수도 확인

In [27]:
np.unique(y_train, return_counts = True)

(array([0, 1, 2]), array([36, 34, 35], dtype=int64))

In [26]:
np.unique(y_test, return_counts = True)

(array([0, 1, 2]), array([14, 16, 15], dtype=int64))

#### 위에서 분리한 학습 데이터를 기반으로 DecisionTreeClassifier를 학습하고, 학습한 모델을 이용해 예측 정확도 측정

붓꽃 데이터는 데이터 양이 크지 않아 전체의 30% 정도인 테스트 데이터가 45개 정도밖에 되지 않아서 이를 통해 알고리즘 성능 예측은 적절하다 할 수 없다.  

학습을 위한 데이터의 양을 일정 수준 이상으로 보장하는 것도 중요하며, 학습된 모델에 대해 다양한 데이터를 기반으로 예측 성능을 평가해 보는것도 중요하다.

In [4]:
dt_clf = DecisionTreeClassifier( )

dt_clf.fit(X_train, y_train)
pred = dt_clf.predict(X_test)
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test,pred)))

예측 정확도: 0.9556


#### 위에서 분리한 학습 데이터를 기반으로 LogisticRegression를 학습하고, 학습한 모델을 이용해 예측 정확도 측정

### logistic regression example


예외적으로 이름 구분  

LinearRegression : 회귀  
LogisticRegression : 분류

In [29]:
from sklearn.linear_model import LogisticRegression

In [30]:
lr = LogisticRegression()

알고리즘에 학습데이터 넣어서 훈련을 시킴

In [31]:
lr.fit(X_train, y_train)

LogisticRegression()

test데이터로 예측을 시킴

In [32]:
lr.predict(X_test)

array([1, 2, 1, 0, 0, 1, 1, 1, 1, 2, 2, 1, 1, 0, 0, 2, 1, 0, 2, 0, 2, 2,
       1, 1, 1, 1, 0, 0, 2, 2, 1, 2, 0, 0, 1, 2, 0, 0, 0, 2, 2, 2, 2, 0,
       1])

test데이터로 예측시킨 답과 원래 정답을 비교

In [33]:
lr.predict(X_test) == y_test

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True, False])

정확도만 바로 출력

In [34]:
lr.score(X_test, y_test)

0.9555555555555556


#### 모델마다 성능이 다 다르기 때문에
#### 데이터와 가장 잘 맞는 모델은 직접 실험을 통해 찾아야 한다.

사이킷런에서 제공하는 데이터 셋이 분류용, 회귀용으로 나뉘듯이 각 데이터에 잘 맞는 모델이 있다